In [131]:
using LinearAlgebra
using Plots
include("polynodes.jl")
include("aux_hfd.jl")
include("hfdTypes.jl")
include("postprocess.jl")

Main.HfdPostProcess

In [132]:
occ = Dict(
    -1 => [2 2 2 2 2 2], #s1/2
    1  => [2 2 2 2],   #p1/2
    -2 => [4 4 4 4],   #p3/2
    2  => [4 4 4],     #d3/2
    -3 => [6 6 6],     #d5/2
    3  => [6], #f5/2
    -4 => [8]
    )
#occ = Dict(-1 => [2 2],
#            1 => [2],
#            -2 => [4])
#occ = Dict(-1 => [2])
N, Nprec=123, 300
Z= 80e0
grid = hfd_funcs.leg_rat_grid(N, 0.5)
grid_prec = hfd_funcs.leg_rat_grid(Nprec, 0.05)
cpars = hfd_funcs.HfdTypes.CalcParams(80., length(grid.xs);scale=1/10)
cpars_prec = hfd_funcs.CalcParams(80., length(grid_prec.xs))
@show(N=length(grid.xs))
occ_block, ztot = hfd_funcs.from_dict(occ, Float64, N)
println("Ztot=$ztot")
@time hfd_funcs.hcore_calc!(cpars, grid, occ_block)
@time hs = hfd_funcs.sc_coul_calc!(cpars, grid, occ_block; tol=1e-5, dump=0.5, maxiter=30);

N = length(grid.xs) = 123
Ztot=80.0
+=========================================================
|Starting calc_occ iterations                             
|atomic charge: 80.0
!atomic scale: 0.1 a.e
|fine structure constant: 0.0072973525643
|number of nodes in radial grid: 123
|maximal iteration number: 3
|tolerance: 1.0e-6
|dumping: 0.5 
|aitken accelerated: false
|non-interacting electrons approximation


+=========================================================
iter no: 1
κ      |δpot|
-4  0.0
-3  0.0
-2  0.0
-1  0.0
1  0.0
2  0.0
3  0.0
orbital energies:
[-6097.696378821576, -5794.12300153193, -5541.408646223093, -5386.4530991025995, -5171.535235742398, -4923.231506827875, -5794.123001529552, -5541.408646222196, -5386.4530991028305, -5171.535235741444, -5792.544153408276, -5525.8005514496135, -5358.651251123794, -6098.236060591995, -5792.544153407396, -5525.800551452265, -5358.651251117721, -6099.911457308181, -5792.440553685274, -5514.17029930455, -5792.4405536870745, -6102.498622

In [127]:
@time hfd_funcs.hded_calc!(cpars, grid, occ_block; tol=1e-3, dump=0.3, maxiter=26, ecp=nothing);
#@time hfd_funcs.hfd_calc!(cpars, grid, occ_block; tol=1e-8, dump=0.8, maxiter=40, ecp=nothing, aitken=true);

+=========================================================
|Starting calc_occ iterations                             
|atomic charge: 80.0
!atomic scale: 0.1 a.e
|fine structure constant: 0.0072973525643
|number of nodes in radial grid: 123
|maximal iteration number: 26
|tolerance: 0.001
|dumping: 0.3 
|aitken accelerated: false
|Dirac Hartree + dominant exchange part


+=========================================================
iter no: 1
κ      |δpot|
-4  0.32890596270008543
-3  1.1332694828089738
-2  0.005181265491807944
-1  0.00593158630981367
1  0.005929928334829371
2  0.005097054566848182
3  0.005935441655533771
orbital energies:
[-3129.6736492681416, -549.6539865015492, -128.95272567902487, -27.464789332084504, -3.795857662268099, -0.06875480229241145, -526.3755301144229, -118.51369151641677, -23.073002098600487, -2.380617200387988, -86.08368375626343, -12.597393695179695, -0.14597419919486176, -450.3515791210652, -102.51691970544907, -19.540229374081104, -1.9460454093979618, -82

In [128]:
include("matutils.jl")
rep = HfdPostProcess.report(cpars, grid, occ_block; sortBy=:rhoav)
ravs = HfdPostProcess.moments(cpars, grid, occ_block, 1)[:, 2]
perm = sortperm(ravs)
klust = Integer.(round.(log.(ravs./ravs[perm[1]])./log(3); digits=0))
#display([rep klust[perm] .+ 1])

klust[end-6]=4
MatUtils.kmeans!(klust, log.(ravs./ravs[perm[1]]))

[rep klust[perm] .+ 1]

22×7 Matrix{Float64}:
 -1.0  1.0  -3128.98      1.0  0.0164401   0.000373576  1.0
  1.0  2.0   -526.207     1.0  0.0566952   0.00401611   2.0
 -2.0  1.0   -450.255     1.0  0.0655679   0.00523113   2.0
 -1.0  2.0   -549.706     1.0  0.0688078   0.00565858   2.0
  2.0  2.0    -86.4358    1.0  0.162866    0.0311149    3.0
 -3.0  1.0    -83.1253    1.0  0.167722    0.0328697    3.0
  1.0  3.0   -118.87      1.0  0.17073     0.0339599    3.0
 -1.0  3.0   -129.362     1.0  0.179816    0.037061     3.0
 -2.0  2.0   -102.89      1.0  0.186996    0.040538     3.0
 -1.0  4.0    -27.8913    1.0  0.399963    0.180581     4.0
  1.0  4.0    -23.4862    1.0  0.403721    0.185426     4.0
 -2.0  3.0    -19.9426    1.0  0.43929     0.219205     4.0
  2.0  3.0    -12.9969    1.0  0.44884     0.232744     4.0
 -3.0  2.0    -12.3867    1.0  0.460197    0.244457     4.0
  3.0  2.0     -3.4827    1.0  0.495263    0.296843     4.0
 -4.0  1.0     -3.38097   1.0  0.501919    0.304819     4.0
 -1.0  5.0     -4.

In [119]:
include("refine_occ.jl")

make_occ_blocks! (generic function with 1 method)

In [129]:
make_occ_blocks!(cpars, grid, occ_block) 

6-element Vector{Float64}:
 0.016440149266931846
 0.06347825999971618
 0.173411660619275
 0.44835548351101184
 1.216265748369473
 3.3202800267580495

  N   A   B
 =============================================================================
   1  1s 1/2( 2.00) 0    3074.23499959  .1201472E+08  14.7632   0.0166   0.0218
   2  2s 1/2( 2.00) 0     550.25220048  .1816226E+07  18.2589   0.0692   0.0895
   3  2p 1/2( 2.00) 0     526.85460071  .1346407E+06  24.7599   0.0570   0.0851
   4  2p 3/2( 4.00) 0     455.15650780  .3297927E+09  25.4082   0.0656   0.0851
   5  3s 1/2( 2.00) 0     133.11331813  .4175199E+06  20.9192   0.1798   0.2274
   6  3p 1/2( 2.00) 0     122.63883542  .3473738E+05  28.6792   0.1704   0.2274
   7  3p 3/2( 4.00) 0     106.54505656  .9058503E+08  29.3388   0.1861   0.2386
   8  3d 3/2( 4.00) 0      89.43676167  .7202867E+06  23.4700   0.1622   0.2386
   9  3d 5/2( 6.00) 0      86.02007223  .1384602E+10  23.4700   0.1670   0.2065
  10  4s 1/2( 2.00) 0      30.64831502  .1048862E+06  23.9526   0.3990   0.4805
  11  4p 1/2( 2.00) 0      26.12403533  .8614877E+04  30.0000   0.4016   0.5026
  12  4p 3/2( 4.00) 0      22.18854415  .2261737E+08  30.0000   0.4340   0.5256
  13  4d 3/2( 4.00) 0      14.79670816  .1948660E+06  29.0088   0.4416   0.5744
  14  4d 5/2( 6.00) 0      14.05254829  .3739032E+09  29.0088   0.4525   0.5256
  15  4f 5/2( 6.00) 0       4.47290743  .3157325E+06  28.6792   0.4767   0.7137
  16  4f 7/2( 8.00) 0       4.31170656  .3771175E+09  29.0088   0.4832   0.5256
  17  5s 1/2( 2.00) 0       5.10305540  .2010366E+05  29.0088   0.9152   1.0349
  18  5p 1/2( 2.00) 0       3.53788277  .1461030E+04  30.0000   0.9871   1.1197
  19  5p 3/2( 4.00) 0       2.84194967  .3658245E+07  30.0000   1.0792   1.1641
  20  5d 3/2( 4.00) 0       0.65006541  .1948808E+05  30.0000   1.4312   1.7419
  21  5d 5/2( 6.00) 0       0.57464856  .3533555E+08  30.0000   1.4987   1.5125
  22  6s 1/2( 2.00) 0       0.32803482  .1701671E+04  30.0000   2.8434   3.2211
  =============================================================================
 -1.0  4.0  -30.6412   1.0  0.399234  0.179964
  1.0  4.0  -26.1396   1.0  0.401624  0.18358
 -2.0  3.0  -22.1918   1.0  0.434225  0.214329
  2.0  3.0  -14.8206   1.0  0.441672  0.225441
 -3.0  2.0  -14.0709   1.0  0.452681  0.236589
  3.0  2.0   -4.5037   1.0  0.476791  0.273419
 -4.0  1.0   -4.33961  1.0  0.483448  0.281584
 
 -1.0  5.0  -5.10376   1.0  0.915551  0.940475
  1.0  5.0  -3.54371   1.0  0.987268  1.10236
 -2.0  4.0  -2.84533   1.0  1.08003   1.32099
  2.0  4.0  -0.656923  1.0  1.43653   2.44069
 -3.0  3.0  -0.580446  1.0  1.50711   2.70216

In [ ]:
@time hfd_funcs.exc_pot(cpars, grid, occ_block, -1)

In [ ]:
@time hfd_funcs.sc_coul(cpars, grid, occ_block, -1)

In [ ]:
res = zeros(160, 160)
@time hfd_funcs.exc_func!(cpars, grid, -1, -1, occ_block.vecs[1], occ_block.occs[1], res)

In [ ]:
include("recp.jl")
using .GausECP
#mercury gatchina sorep
lblk = ECPL{Float64}[]
soblk = ECPL{Float64}[]
Nel=60
GausECP.perm = [3, 2, 1]
@ecp_blk lblk begin
 @_r -0.1533054417518667      0   1.241333580354307     
 @_r -2.395038143828458      1   7607.844317587697     
 @_r -3.083263090901668      1   841.1986644533526     
 @_r -20.66235084776708      1   198.9245934440317     
 @_r 43.47025508227014      1   141.3340650803564     
 @_r -41.42648254134264      1   92.24922902157708     
 @_r 10.70202499663918      1   57.53558485747016     
 @_r -9.572044818027125      1   37.05658452036516     
 @_r -17.44928603918218      1   11.09672300653557     
 @_r -4.208396455548661      1   5.176912498322142     
 @_r -1.367889736495112      1   2.774923794909799     
 @_r -0.4243813243216499E-003 1  0.1483608850844247     
 @_r -0.5929678514352788E-005 1  0.6866747702550722E-002
end
@ecp_blk lblk begin
     @_r 10.15330544175187       0   75.45042348565966   
     @_r 48.25843608738218       1   5.516150757473351   
     @_r 480.6203389985901       2   51.33814200088347   
    @_r -262.7712529434285       2   21.42160330126189   
     @_r 984.4925014757564       2   15.02174622132596   
    @_r -1185.343491168202       2   10.41942310721764   
     @_r 895.8591924660080       2   7.044823824714383   
    @_r -446.6972167857539       2   4.654867356257943   
     @_r 187.8018689733064       2   3.105341217453005   
    @_r -77.40249900076539       2   2.110106791683100   
     @_r 28.57866221128257       2   1.397719288132955   
    @_r -10.31101877476388       2  0.9439204802242250   
     @_r 2.910507016043967       2  0.6346119391590400   
   @_r -0.5716441707929951       2  0.4195337735684076   
    @_r 0.5859843273819665E-001  2  0.2726300506225667   
end
@ecp_blk lblk begin
   @_r 9.153305441751867       0   17.82507436921039        
   @_r 7.620198745806825       1   30.37695602101245        
   @_r 40.16962543708902       1   2.310740519807635        
  @_r -164.0019177645640       2   17.82504671620828        
   @_r 474.2514077674568       2   12.10181863115705        
  @_r -643.4432423810800       2   8.036404872508950        
   @_r 588.4002403856621       2   5.818624035188340        
  @_r -227.4981410970566       2   4.100987052252767        
  @_r -11.32937197613574       2   2.216070578260750        
  @_r -2.859429122040108       2   1.333249311891608        
  @_r 0.7947134802086527       2  0.7455844056608246        
 @_r -0.2229856343783364       2  0.4124123361330905        
  @_r 0.5839600759664158E-001  2  0.2194203883781155        
 @_r -0.1537771398331559E-001  2  0.1230722448993955     
  @_r 0.2430273117120762E-002  2  0.7296494669739809E-001
end
@ecp_blk lblk begin
   @_r 7.153305441751867      0   23.21377576628131      
  @_r -2.144639474061312      1   26.10780773207126      
   @_r 65.89001575081436      1   6.951131994354646      
  @_r -15.42058859523037      1   4.403942853245356      
  @_r -51.58323733116725      2   5.100216264970559      
   @_r 110.8021601784391      2   3.219262698530238      
  @_r -54.95719791070449      2   2.300462248784210      
   @_r 11.76944480623374      2   1.528087159313657      
  @_r -1.304739967169114      2  0.8115270441625856      
  @_r 0.2393217597580563      2  0.4291099486615543      
 @_r -0.4535234615374955E-001 2  0.2254312599912531      
  @_r 0.7747797598180014E-002 2  0.8402807013053894E-001 
 @_r -0.3601488445243359E-002 2  0.4516649928935540E-001 
  @_r 0.1119859476515526E-002 2  0.2504140674511687E-001 
 @_r -0.1821995770439212E-003 2  0.1452539607011488E-001 
end
@ecp_blk lblk begin
    @_r 4.153305441751867     0  0.6768910451935157      
    @_r 59.34201069372035     1   88.91809768600156      
   @_r -37.60918087587240     2   1086.775014787427      
   @_r -79.56194030509143     2   675.5596416277371      
   @_r -335.0124022349072     2   165.4780737519513      
    @_r 183.6728097442637     2   32.45662920610522      
    @_r 165.4744951110425     2   9.126231853126143      
   @_r -136.0311282782796     2   6.012492259965792      
    @_r 143.0064886699650     2   3.505612533112329      
   @_r -113.9945474034395     2   2.178173862269087      
    @_r 121.4376695804941     2   1.294122910515299      
   @_r -73.06442644429838     2  0.9322969459551654      
    @_r 12.08767938333994     2  0.6768931808674640      
end
@ecp_blk soblk begin
    @_r 2.33399457129762  1     30.37695602101245  
   @_r -1.54926388536551  1      2.31074051980764  
  @_r -38.17988766177815  2     17.82504671620828  
   @_r 82.79783496886466  2     12.10181863115705  
  @_r -58.85347280654781  2      8.03640487250895  
  @_r -10.37959113417576  2      5.81862403518834  
   @_r 39.37189964009465  2      4.10098705225277  
   @_r -9.08333857917080  2      2.21607057826075  
    @_r 3.69705947137211  2      1.33324931189161  
   @_r -1.06322690698226  2      0.74558440566082  
    @_r 0.30715900767899  2      0.41241233613309  
   @_r -0.08396259856007  2      0.21942038837812  
    @_r 0.02228051512489  2      0.12307224489940  
   @_r -0.00351899693552  2      0.07296494669740  
end
@ecp_blk soblk begin
  @_r -0.41733455916356  1     26.10780773207126   
   @_r 0.97274463182258  1      6.95113199435465   
  @_r -0.52375725997839  1      4.40394285324536   
  @_r -3.74556386224486  2      5.10021626497056   
   @_r 2.56937879347424  2      3.21926269853024   
   @_r 0.95373846965646  2      2.30046224878421   
  @_r -0.88965926027939  2      1.52808715931366   
   @_r 0.21325692813968  2      0.81152704416259   
  @_r -0.05367393973439  2      0.42910994866155   
   @_r 0.01205720847536  2      0.22543125999125   
  @_r -0.00233175961945  2      0.08402807013054   
   @_r 0.00112079580463  2      0.04516649928936   
  @_r -0.00035450482483  2      0.02504140674512   
   @_r 0.00005882336947  2      0.01452539607011   
end
@ecp_blk soblk begin
   @_r 0.00810962564509  1     88.91809768600156   
  @_r -0.16013674443739  2   1086.77501478742693   
  @_r -0.17281127425240  2    675.55964162773705   
  @_r -0.06535456696494  2    165.47807375195131   
  @_r -0.03967362070847  2     32.45662920610522   
   @_r 0.21851422563780  2      9.12623185312614   
  @_r -0.21083072366221  2      6.01249225996579   
   @_r 0.25824759797377  2      3.50561253311233   
  @_r -0.45339487218633  2      2.17817386226909   
   @_r 0.24352517571891  2      1.29412291051530   
  @_r -0.06258006221000  2      0.93229694595517   
   @_r 0.01631215943744  2      0.67689318086746   
end
ecp = RECP.ECPnum(Nel*0e0, lblk, soblk);

In [ ]:
grid_ecp = hfd_funcs.leg_rat_grid(250, 1e0)
cpars_ecp = hfd_funcs.CalcParams(20e0, length(grid_ecp.xs), 1e-6)

In [ ]:
occ_ecp = Dict(
    -1 => [2 2], #s1/2
    1 => [2],    #p1/2
    -2 => [4],   #p3/2
    2 => [4],    #d3/2
    -3 => [6]) #5s2 5p6 5d10 6s2
occ_block_ecp, ztot=hfd_funcs.from_dict(occ_ecp, Float64, length(grid_ecp.xs))
occ_block_ecp.inds[occ_block_ecp.ks .>0] .-= 1;

In [ ]:
hfd_funcs.hcore_calc!(cpars_ecp, grid_ecp, occ_block_ecp; ecp=ecp)
hfd_funcs.sc_coul_calc!(cpars_ecp, grid_ecp, occ_block_ecp; ecp=ecp, tol=1e-4, dump=0.3);

In [ ]:
hfd_funcs.hfd_calc!(cpars_ecp, grid_ecp, occ_block_ecp;ecp=ecp, dump=0.7, maxiter=20, tol=1e-8);

In [ ]:
HfdPostProcess.report(cpars_ecp, grid_ecp, occ_block_ecp)

In [ ]:
-1.0  5.0     -5.08949   1.0  0.915576   0.940498
 -2.0  4.0     -2.83016   1.0  1.07983    1.32033
  1.0  5.0     -3.52905   1.0  0.987247   1.10224
 -3.0  3.0     -0.566696  1.0  1.51386    2.91261
  2.0  4.0     -0.643041  1.0  1.43354    2.42879
 -1.0  6.0     -0.325123  1.0  2.85594    9.44103

In [ ]:
argmax([0 1 0;
        2 -1 0;
        0 0  0])[1]

In [ ]:
@time hfd_funcs.exc_pot(cpars, grid, occ_block, -3)

In [ ]:
@time lhs, rhs = hfd_funcs.dirac_h1(cpars, grid, -3)

In [ ]:
@time mat = lhs .+ hfd_funcs.sc_coul(cpars, grid, occ_block, -3)
@time mat = lhs .+ hfd_funcs.exc_pot(cpars, grid, occ_block, -3)
@time eigen(mat, rhs)

In [ ]:
@time hfd_funcs.hfd_calc!(cpars, grid, occ_block; tol=1e-8, dump=0.7, maxiter=1, ecp=nothing)

In [ ]:
function get_states(cpars, grid, occ_block, ks, inds; pot_func, ecp=nothing)
    κs = sort(unique(ks))
    lhs = zeros(eltype(grid.xs), 2*cpars.N, 2*cpars.N)
    rhs = zeros(eltype(grid.xs), size(lhs))
    vecs = zeros(eltype(grid.xs), 2*cpars.N, length(inds))
    ens = zeros(eltype(grid.xs), length(inds))
    for κ in ks
        mask = κ .== ks
        hfd_funcs.dirac_h1!(cpars, grid, κ, lhs, rhs)
        @views lhs .+= pot_func(cpars, grid, occ_block, κ)
        if ecp!=nothing
            ecpval = ecp(cpars, grid, κ)
            @views lhs[diagind(lhs)] .+= [ecpval; ecpval.*(cpars.alpha^2*cpars.Z^2)]
        end
        vals, aux = eigen(lhs, rhs)
        for v_ii in findall(mask)
           # println("v_ii is $v_ii")
            els = real(vals) .> -1e0
            @views vecs[:, v_ii] .= hfd_funcs.normalize!(cpars, grid, aux[:, els][:, inds[v_ii]], κ, 
                    hfd_funcs.rcut(vals[els][inds[v_ii]]))
            @views ens[v_ii] = vals[els][inds[v_ii]]*cpars.Z^2
        end
    end
    ens, vecs
end
        
            
            

In [ ]:
ens, vecs = get_states(cpars, grid, occ_block, [-1, -1, -1, -1, -1], [5, 6, 7, 8, 16];
    pot_func = hfd_funcs.hfd_pot)
ens_ecp, vecs_ecp = get_states(cpars_ecp, grid, occ_block_ecp, [-1, -1, -1], [1, 2, 3]; 
    pot_func=hfd_funcs.hfd_pot, ecp=ecp)

In [ ]:
HfdPostProcess.report(cpars, grid, occ_block)

In [ ]:
include("matutils.jl")
vec_prec = MatUtils.project(grid, grid_prec, PolyNodes.Lagrange(grid.ts, occ_block.vecs[1:length(grid.xs), 1]))

In [ ]:
[grid.xs grid_prec.xs[1:80]]

In [ ]:
@time oc_block_prec = hfd_funcs.sh_blk_new_grid(cpars, grid, grid_prec, occ_block);
@time hfd_funcs.hfd_calc!(cpars_prec, grid_prec, oc_block_prec; tol=1e-8, dump=0.8, maxiter=5, ecp=nothing);

In [ ]:
@time oc_block_tst = hfd_funcs.sh_blk_new_grid(cpars_prec, grid_prec, grid, oc_block_prec);

In [ ]:
rcut(en) = log(1e10)/sqrt(2*abs(en))
function sh_blk_new_grid(cpars, old_grid, new_grid, occ_block)
    ks = occ_block.ks
    inds = occ_block.inds
    occs = occ_block.occs
    ens = occ_block.ens
    vecs  = zeros(eltype(new_grid.xs), 2*length(new_grid.xs), length(occ_block.ks))
    pqs = reshape(vecs, :, 2, length(occ_block.ks))
    old_pqs = reshape(occ_block.vecs, :, 2, length(occ_block.ks))
    for kk=1:length(ks), i_pq=1:2
        func = PolyNodes.Lagrange(old_grid.ts, old_pqs[:, i_pq, kk])
        @views pqs[:, i_pq, kk] .= hfd_funcs.HfdTypes.project(old_grid, new_grid, func)
        xmask = new_grid.xs .> rcut(ens[kk]/cpars.Z^2)
        pqs[xmask, i_pq, kk] .= 0e0
    end
    hfd_funcs.HfdTypes.ShellBlock(ks, occs, inds, vecs, ens)
end

In [ ]:
as_pqs(occ_block) = reshape(occ_block.vecs, :, 2, length(occ_block.ks))
pqs1 = as_pqs(occ_block)
pqs2 = as_pqs(oc_block_prec)
scatter(grid.xs, pqs1[:, :, 1]; xaxis =:log)
plot!(grid_prec.xs, pqs2[:, :, 1])

In [ ]:
grid_cutted = hfd_funcs.leg_exp_grid(90, 1e2)
cpars_cutted = hfd_funcs.CalcParams(80e0, length(grid_cutted.xs))
cpot = diag(hfd_funcs.coul_pot(cpars, grid, occ_block, -1))[1:N]
blk_cutted = hfd_funcs.sh_blk_new_grid(cpars, grid, grid_cutted, occ_block)
lhs, rhs = hfd_funcs.dirac_h1(cpars_cutted, grid_cutted, -1)
new_pot = hfd_funcs.HfdTypes.project(grid, grid_cutted, PolyNodes.Lagrange(grid.ts, cpot))
exc = hfd_funcs.exc_pot(cpars_cutted, grid_cutted, blk_cutted, -1)
lhs .+= diagm([new_pot; (cpars_cutted.alpha*cpars_cutted.Z)^2 .* new_pot]) .+ exc
vals, vecs = eigen(lhs, rhs)
mask = (real(vals) .> -1e0) .&& (abs.(imag(vals)) .< 1e-10)
vals[mask].*80^2

In [ ]:
grid_prec = hfd_funcs.leg_rat_grid(Nprec, 1e1)
@time occ_prec = sh_blk_new_grid(cpars, grid, grid_prec, occ_block)
pqs = reshape(occ_block.vecs, :, 2, size(occ_block.vecs, 2))
pqs_prec =  reshape(occ_prec.vecs, :, 2, size(occ_prec.vecs, 2))

In [ ]:
plot(grid_prec.xs, pqs_prec[:, :, 1]; xaxis=:log, ls=:dash)
plot!(grid.xs, pqs[:, :, 1]; ls=:dash)

In [ ]:
typeof(occ_block)

In [ ]:
include("inverse_iter.jl")
kmax = 30
occ_prec = hfd_funcs.sh_blk_new_grid(cpars, grid, grid_prec, occ_block)
green_funcs = zeros(eltype(grid_prec.xs), cpars_prec.N, cpars_prec.N, kmax+1)
@time for kk=0:kmax
    green_funcs[:, :, kk+1] .= hfd_funcs.HfdTypes.potk_mat(grid_prec, kk)
end


In [ ]:
@time cpot = RadInts.coulpot_func(cpars_prec, grid_prec, occ_prec;green_funcs=green_funcs)
st = occ_prec.vecs[:, 6]
kappa1 = -1
@time res=similar(st)
@time RadInts.twoelint!(cpars_prec, grid_prec, st, kappa1, occ_prec, res; cpot=cpot, green_funcs=green_funcs)

In [ ]:
green_funcs[:, :, end]
plot(grid_prec.xs, cpot; xaxis=:log)

In [ ]:
cpot

In [ ]:
tmp = zeros(eltype(grid_prec.xs), 2cpars_prec.N)
RadInts.twoelint!(cpars_prec, grid_prec, occ_prec.vecs[:,6], -1, occ_prec, tmp; 
    cpot=cpot, green_funcs=green_funcs)
@show cpars_prec.N

In [ ]:
plot(grid_prec.xs, tmp[1:cpars_prec.N] .- cpot .* occ_prec.vecs[1:cpars_prec.N, 6])
plot!(grid_prec.xs[1:100], cpot[1:100]; xaxis=:log)

In [ ]:
include("inverse_iter.jl")
kappa =-1
lhs, rhs = hfd_funcs.dirac_h1(cpars, grid, kappa)
an2 = (cpars.alpha*cpars.Z)^2 # guess what
for ii=1:cpars.N
    lhs[ii, ii] += cpot[ii]
    lhs[cpars.N+ii, cpars.N + ii] += cpot[ii] * an2
end


In [ ]:
occ_prec = hfd_funcs.sh_blk_new_grid(cpars, grid, grid_prec, occ_block)
@time InvIter.hfd_iter(cpars_prec, grid_prec, 7, occ_prec, cpot, green_funcs; dump=0.5, niter=10)

In [ ]:
using Profile
Profile.Allocs.clear()
Profile.Allocs.@profile InvIter.hfd_iter(cpars, grid, 1, occ_block, cpot, green_funcs; dump=0.5, niter=10)
Profile.Allocs.print(;format=:flat)

In [ ]:
plot(grid.xs, grid.xs.^1.6 .* reshape(occ_block.vecs, cpars.N, 2, :)[:, :, 1]; xaxis=:log)

In [ ]:
gam = sqrt(1 - (cpars.alpha*cpars.Z)^2)
dot(grid.ws, grid.xs.^(2gam) .* occ_block.vecs[1:cpars.N, 1].^2)

In [ ]:
gam = sqrt(1^2 - (cpars.alpha*cpars.Z)^2)
rho = (grid_prec.xs.^gam.*occ_prec.vecs[1:cpars_prec.N, 1]).^2 + (grid_prec.xs.^gam.*occ_prec.vecs[cpars_prec.N+1:end, 1]).^2 .*(cpars.alpha*cpars.Z)
grid_prec.xs.*(grid_prec.pot*rho)

In [ ]:
[grid.xs occ_block.vecs[1:cpars.N]][60:80, :]

In [ ]:
grid_prec.xs

In [39]:
1 in [1 2 3 5]

true

In [71]:
[grid.xs hfd_funcs.δnuc(cpars, grid)]

123×2 Matrix{Float64}:
    4.73989e-5   -2.30996e-6
    0.000249809  -5.7817e-35
    0.000614237  -2.47443e-87
    0.00114123   -2.34912e-163
    0.00183149   -4.47123e-263
    0.00268592   -0.0
    0.00370562   -0.0
    0.00489193   -0.0
    0.00624641   -0.0
    0.00777084   -0.0
    0.00946723   -0.0
    0.0113378    -0.0
    0.0133852    -0.0
    ⋮            
   22.05         -0.0
   26.4069       -0.0
   32.1715       -0.0
   40.023        -0.0
   51.1045       -0.0
   67.4651       -0.0
   93.078        -0.0
  136.501        -0.0
  219.061        -0.0
  407.009        -0.0
 1000.76         -0.0
 5274.38         -0.0

In [63]:
grid.xs

123-element Vector{Float64}:
    4.739890711717106e-5
    0.00024980941025495293
    0.0006142365942686779
    0.0011412336281919253
    0.0018314943812667973
    0.0026859187175469045
    0.0037056209413018314
    0.004891933999958964
    0.006246413493744823
    0.007770842128489069
    0.00946723476966267
    0.011337844160515595
    0.01338516734500812
    ⋮
   22.050047298289236
   26.406866004962144
   32.17154535715808
   40.02296681933609
   51.10453248185716
   67.46507642310868
   93.07802144821959
  136.50055526083807
  219.06119292691147
  407.0092897960785
 1000.7629406152626
 5274.383212727546

In [98]:
diag(hfd_funcs.coul_pot(cpars, grid, occ_block, -1))[1:cpars.N]

123-element Vector{Float64}:
 1.8471247919718192e-5
 9.735003187586467e-5
 0.00023936519987878668
 0.0004447289637464622
 0.0007137043122729597
 0.0010466285061006663
 0.0014439142827148569
 0.001906049218837436
 0.002433594887699627
 0.003027186002691378
 0.0036875296522268946
 0.004415404594045788
 0.0052116606585377485
 ⋮
 1.6847101548073085
 1.7476327794129312
 1.8078026941863237
 1.8645946840833014
 1.9138618436833774
 1.9516280035667704
 1.9769705676989053
 1.990850673519179
 1.995989167920887
 1.997486601408328
 1.9996987119258767
 2.0